Full Credits to Farhan for the vast majority of the pre-processing/actual machine learning code.

In [ ]:
!unzip ../input/dogs-vs-cats-redux-kernels-edition/train.zip -d train

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import cv2
import pickle
from sklearn.model_selection import train_test_split
import os, os.path, shutil
import numpy as np




In [ ]:
### Sorting into different folders
path = '../working/train/train'
images = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

for image in images:
    folder_name = image.split('.')[0]

    new_path = os.path.join(path, folder_name)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    old_image_path = os.path.join(path, image)
    new_image_path = os.path.join(new_path, image)
    shutil.move(old_image_path, new_image_path)


In [ ]:
path = '/kaggle/working/train/train'
catagories = os.listdir(path)
print(catagories)



In [ ]:

X =[]
y = []

def create_data(img_size):
    for c in catagories:
        curr_path = os.path.join(path, c)
        classification = catagories.index(c)
        print(classification)
        for img in os.listdir(curr_path):
            try:
                img_array = cv2.imread(os.path.join(curr_path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (img_size, img_size))
                X.append(new_array)
                y.append(classification)
            except Exception as e:
                pass
            


In [ ]:
create_data(50)


In [ ]:
X = np.array(X).reshape(-1,50,50,1)
X = tf.keras.utils.normalize(X, axis = 1)
y = np.array(y)

In [ ]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.1, random_state = 0)
unique, counts = np.unique(yTrain, return_counts=True)
dict(zip(unique, counts))

In [ ]:
print(xTrain.shape[1:])

In [ ]:
yTest = tf.keras.utils.to_categorical(yTest, 2)
yTrain = tf.keras.utils.to_categorical(yTrain, 2)

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3,3),input_shape = xTrain.shape[1:], activation = 'relu'))

model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(128, activation = 'sigmoid'))

model.add(Dense(2, activation="sigmoid"))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.fit(xTrain, yTrain, batch_size = 32, epochs = 10)

In [ ]:
 model.evaluate(xTest, yTest)

In [ ]:
import sklearn.metrics as metrics


probs = model.predict_proba(xTest)
print(probs)

fpr, tpr, threshold = metrics.roc_curve(y_true = np.ravel(yTest), y_score = np.ravel(probs), pos_label = 1)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
